In [1]:
# %load ../kiwi.py
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
#%load_ext rpy2.ipython

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
from rpy2.robjects import r, pandas2ri
import os 
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances,cosine_similarity

#Set environment variables


# Set up the local source files
#TOP = os.getcwd().replace('notebooks','')
TOP = "/home/ishah/ipynb/Camda18/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

CMAP_DIR= '/share/home/ishah/projects/HTTR/data/'
CMAP_DAT= '/mnt/data/CMap/CMap2.0/CEL1/'

from db.mongo import *
#DB=openMongo(db='httr_ph1',host='pb.epa.gov')
#DB1=openMongo(db='httr_v1',host='pb.epa.gov')
#MSG=openMongo(db='msigdb_v6',host='pb.epa.gov')
CMP=openMongo(db='cmap_v2',host='localhost')
#CMP = pymongo.MongoClient("mongodb://localhost/cmap_v2")['cmap_v2']
#from gexp.deseq2 import *
pd.options.display.max_colwidth = 500


Populating the interactive namespace from numpy and matplotlib


In [5]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [14]:
#%%R
#library(hgu133a.db)

# Goal 

Calculate the perturbations for each treatment and consider each treatment separately. For each pert_id

* Build DF with trt and controls
* Calcuate FC, Amplitude
* Consider/exclude outliers




## Get Treatments and controls

In [9]:
CMP.collection_names()

[u'cmap_trt_info',
 u'chembl_mol',
 u'chembl_activity',
 u'cmap1',
 u'cmap_tgt_b',
 u'gene_corr',
 u'chembl_mech',
 u'genes1',
 u'gene_freq',
 u'cmap_tgt_a',
 u'cmap_tgt_new',
 u'cmap_affy_raw',
 u'cmap_fc']

In [ ]:
K = [u'instance_id',u'scanner', u'name',
     u'gexp_array','cell',
     'pert_id','conc','timeh']

CMP.cmap_fc.drop()

for P in CMP.cmap_trt_info.find(no_cursor_timeout=True):
    P_scans = []
    ScIDs=[P['pert_id']]+P['ctrl_ids']
    
    for X in CMP.cmap_affy_raw.find(dict(scan_id={'$in':ScIDs}),
                                    dict(_id=0,scan_id=1,probe_levels=1)):
        Y = X.pop('probe_levels')
        X.update(Y)
        X['trt']= 1 if X['scan_id']==P['pert_id'] else 0
        
        P_scans.append(X)
    DF = pd.DataFrame(P_scans).set_index(['trt']).drop('scan_id',axis=1)
    
    X_chem = DF.query("trt==1")
    X_ctrl = DF.query("trt==0").median(axis=0)
    L2FC = X_chem-X_ctrl
    
    Y = {k:P[k] for k in K}
    Y['fc1']=L2FC.iloc[0].to_dict()
    
    CMP.cmap_fc.insert_one(Y)
    

In [8]:
CMP.cmap_fc.count()

6100

In [24]:
X_chem.shape

(1, 22284)

In [27]:
def getBounds(X):
    q75, q25 = np.percentile(X.dropna(), [75 ,25])
    iqr = q75 - q25

    return q25 - (iqr*1.5),q75 + (iqr*1.5)

def remOutlier(X):
    IQR = getBounds(X)
    X1 = X[np.logical_or(X<IQR[0],X>IQR[1])]
    return X1

def meanRemOutlier(X):
    return np.mean(remOutlier(X))

In [30]:
zip(X_ctrl.iloc[:,:20].apply(meanRemOutlier),
    X_ctrl.iloc[:,:20].mean(axis=0),
    X_ctrl.iloc[:,:20].median(axis=0),
   )

[(7.268106503189644, 7.142139356024288, 7.107853833347347),
 (nan, 7.531686750799236, 7.49729251545941),
 (nan, 6.202211517454833, 6.236951386844174),
 (nan, 7.564305947595896, 7.541432547299573),
 (nan, 4.14034494677769, 4.116683384206098),
 (7.251219332413167, 6.941575088923008, 6.89730412230967),
 (4.917972483154669, 5.060743371499148, 5.073815140039253),
 (nan, 4.705784150132551, 4.717288880630234),
 (nan, 9.362550840655175, 9.282272216926222),
 (nan, 3.4271680144703502, 3.455938657097506),
 (nan, 6.628762161629034, 6.66758674494729),
 (nan, 7.479944868006287, 7.420195406611283),
 (6.648196172783096, 6.62450182122568, 6.598226111236895),
 (7.631825823424513, 7.80339342415869, 7.822775803408004),
 (nan, 7.059523521747616, 7.072117758072451),
 (nan, 7.089580201656561, 7.0659468464785515),
 (5.866029013562128, 5.767906255519081, 5.748505494296783),
 (nan, 5.775752036761175, 5.803355332516707),
 (nan, 8.336601338976283, 8.316478230295408),
 (nan, 5.8734778380713974, 5.887417314376726)]